### problem 2.4

In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn

In [2]:
training_data = pd.read_csv('iris-train.csv')
test_data = pd.read_csv('iris-test.csv')

In [3]:
training_data['label'] = training_data.apply(lambda x: 1 if x.label == 3 else 0, axis=1)
test_data['label'] = test_data.apply(lambda x: 1 if x.label == 3 else 0, axis=1)

In [4]:
training_data.head()

,sepal,petal,label
0,0.62963,0.266670,0
1,0.66667,0.133330,0
2,0.72549,0.266670,0
3,0.76364,0.142860,0
4,0.63265,0.066667,0


In [5]:
training_label = torch.from_numpy(training_data['label'].to_numpy().reshape(-1, 1)).float()
training_feature = torch.from_numpy(training_data[['sepal','petal']].to_numpy()).float()
test_label = torch.from_numpy(test_data['label'].to_numpy().reshape(-1, 1)).float()
test_feature = torch.from_numpy(test_data[['sepal','petal']].to_numpy()).float()

In [6]:
training_feature.shape[0]

120

In [7]:
def gd(params, lr):
    for param in params:
        param.data -= lr * param.grad

In [8]:
def linear(X, w, b):
    return torch.mm(X, w) + b

In [9]:
def loss(y_pred, y_true, cfp, cfn):
    bceloss = nn.BCELoss()
    ret = torch.tensor(0, dtype=torch.float32)
    for i in range(len(y_pred)):
        if y_true[i] == 1:
            ret += cfn * bceloss(y_pred[i], y_true[i])
        else:
            ret += cfp * bceloss(y_pred[i], y_true[i])
    return ret

In [10]:
def false(y_pred, y_true):
    fp = 0
    fn = 0
    for i in range(len(y_pred)):
        if y_pred[i] > 0.5 and y_true[i] == 0:
            fp += 1
        elif y_pred[i] <= 0.5 and y_true[i] == 1:
            fn += 1
    return fp, fn

In [15]:
w = torch.zeros((2, 1), dtype=torch.float32, requires_grad=True)
b = torch.zeros((1, 1), dtype=torch.float32, requires_grad=True)
iterations = 5000
step_size = 0.03
cfp = 1.0
cfn = 20.0
sig = nn.Sigmoid()

In [16]:
X = training_feature
y = training_label
train_l = []
train_l.append(loss(sig(linear(X, w, b)), y, cfp, cfn).item())
print('number of iteration: %d, R(p): %f' % (0, train_l[-1]))
for it in range(iterations):
    l = loss(sig(linear(X, w, b)), y, cfp, cfn)
    l.backward()
    gd([w, b], step_size)
    w.grad.data.zero_()
    b.grad.data.zero_()
    if (it+1) % 100 == 0:
        train_l.append(loss(sig(linear(X, w, b)), y, cfp, cfn).item())
        print('number of iteration: %d, R(p): %f' % (it + 1, train_l[-1]))

number of iteration: 0, R(p): 609.969299
number of iteration: 100, R(p): 136.014862
number of iteration: 200, R(p): 125.887482
number of iteration: 300, R(p): 121.078384
number of iteration: 400, R(p): 118.112206
number of iteration: 500, R(p): 116.092987
number of iteration: 600, R(p): 114.641602
number of iteration: 700, R(p): 113.559692
number of iteration: 800, R(p): 112.731285
number of iteration: 900, R(p): 112.083488
number of iteration: 1000, R(p): 111.568268
number of iteration: 1100, R(p): 111.152695
number of iteration: 1200, R(p): 110.813637
number of iteration: 1300, R(p): 110.534164
number of iteration: 1400, R(p): 110.301872
number of iteration: 1500, R(p): 110.107323
number of iteration: 1600, R(p): 109.943443
number of iteration: 1700, R(p): 109.804497
number of iteration: 1800, R(p): 109.686249
number of iteration: 1900, R(p): 109.585068
number of iteration: 2000, R(p): 109.498199
number of iteration: 2100, R(p): 109.423340
number of iteration: 2200, R(p): 109.358696


In [17]:
print(f'false positive: %d, false negative: %d' % false(sig(linear(X, w, b)), y))

false positive: 34, false negative: 0


### problem 2.5

In [18]:
subinterval = [0 for k in range(10)]
sumsubinterval = [0 for k in range(10)]
for i in range(training_feature.shape[0]):
    prob = sig(linear(training_feature[i].reshape(1,-1), w, b))
    for j in range(10):
        if prob.item() < 0.1*(j+1):
            if training_label[i] == 1:
                subinterval[j] += 1
            sumsubinterval[j] += 1
            break
for i in range(10):
    if sumsubinterval[i] > 0:
        subinterval[i] /= sumsubinterval[i]
print(subinterval)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.2727272727272727, 0.7346938775510204]
